I'm experimenting with a new, smaller increment method here of working through the TIR registrations on WoRMS matching. In this process, we work on WoRMS matching until there is nothing left to work on by using a while loop on the number of possible records to work on. I also threw in a safeguard total number to process and include that in the check so the loop doesn't run away on us.

This might be a little less efficient than grabbing up a whole batch of records at once that meet some criteria and then processing through all of those. It has to execute the following three API interactions:

1) Get a single TIR registration that does not currently have any WoRMS data and has been processed by ITIS (so we have an alternative name to work against)
2) Check the TIR registered name against the WoRMS REST service
3) If we find a match, insert the WoRMS information into the TIR

However, that is only a single additional API interaction per record, and it allows us to simply kick off this process whenever and have it run until there's nothing left to do. It seems like that might be conducive to processing on the Kafka/microservices architecture.

In [14]:
import requests,re
from IPython.display import display
from bis import worms
from bis import bis
from bis import tir
from bis2 import gc2

In [19]:
numberWithoutWoRMS = 1
totalRecordsToProcess = 50
totalRecordsProcessed = 0

while numberWithoutWoRMS == 1 and totalRecordsProcessed <= totalRecordsToProcess:
    q_recordToSearch = "SELECT id, \
        registration->'scientificname' AS scientificname, \
        itis->'nameWInd' AS itisNameWInd \
        FROM tir.tir \
        WHERE worms IS NULL \
        AND itis IS NOT NULL \
        LIMIT 1"
    recordToSearch  = requests.get(gc2.sqlAPI("DataDistillery","BCB")+"&q="+q_recordToSearch).json()
    
    numberWithoutWoRMS = len(recordToSearch["features"])

    if numberWithoutWoRMS == 1:
        tirRecord = recordToSearch["features"][0]

        # Set up a local data structure for storage and processing
        thisRecord = {}
        
        # Set data from query results
        thisRecord["id"] = tirRecord["properties"]["id"]
        thisRecord["scientificname_submitted"] = tirRecord["properties"]["scientificname"]
        thisRecord["scientificname_search"] = bis.cleanScientificName(thisRecord["scientificname_submitted"])
        thisRecord["itisNameWInd"] = tirRecord["properties"]["itisnamewind"]

        # Set defaults for thisRecord
        thisRecord["matchMethod"] = "Not Matched"
        thisRecord["matchString"] = thisRecord["scientificname_search"]
        wormsData = 0

        # Handle the cases where there is enough interesting stuff in the scientific name string that it comes back blank from the cleaners
        if len(thisRecord["scientificname_search"]) != 0:
            try:
                wormsSearchResults = requests.get("http://www.marinespecies.org/rest/AphiaRecordsByName/"+thisRecord["scientificname_search"]+"?like=false&marine_only=false&offset=1").json()
                thisRecord["matchMethod"] = "Exact Match"
                wormsData = wormsSearchResults[0]
            except:
                try:
                    wormsSearchResults = requests.get("http://www.marinespecies.org/rest/AphiaRecordsByName/"+thisRecord["scientificname_search"]+"?like=true&marine_only=false&offset=1").json()
                    thisRecord["matchMethod"] = "Fuzzy Match"
                    wormsData = wormsSearchResults[0]
                except:
                    if thisRecord["itisNameWInd"] != None and thisRecord["itisNameWInd"] != thisRecord["scientificname_search"]:
                        try:
                            wormsSearchResults = requests.get("http://www.marinespecies.org/rest/AphiaRecordsByName/"+thisRecord["itisNameWInd"]+"?like=false&marine_only=false&offset=1").json()
                            thisRecord["matchMethod"] = "ITIS Name Match"
                            wormsData = wormsSearchResults[0]
                        except:
                            pass

        thisRecord["wormsPairs"] = worms.packageWoRMSPairs(thisRecord["matchMethod"],wormsData)
        display (thisRecord)
        print (tir.cacheToTIR(gc2.sqlAPI("DataDistillery","BCB"),thisRecord["id"],"worms",thisRecord["wormsPairs"]))
        totalRecordsProcessed = totalRecordsProcessed + 1


{'id': 20402,
 'itisNameWInd': 'Acaena exigua',
 'matchMethod': 'Not Matched',
 'matchString': 'Acaena exigua',
 'scientificname_search': 'Acaena exigua',
 'scientificname_submitted': 'Acaena exigua',
 'wormsPairs': '"cacheDate"=>"2017-06-26T13:53:49.507084","wormsMatchMethod"=>"Not Matched"'}

{'_execution_time': 0.067, 'affected_rows': 1, 'success': True, 'auth_check': {'success': True, 'session': None, 'auth_level': None}}


{'id': 20381,
 'itisNameWInd': None,
 'matchMethod': 'Exact Match',
 'matchString': 'Abacion wilhelminae',
 'scientificname_search': 'Abacion wilhelminae',
 'scientificname_submitted': 'Abacion wilhelminae',
 'wormsPairs': '"cacheDate"=>"2017-06-26T13:53:51.147335","wormsMatchMethod"=>"Exact Match","AphiaID"=>"944219","scientificname"=>"Abacion wilhelminae","status"=>"accepted","rank"=>"Species","valid_name"=>"Abacion wilhelminae","valid_AphiaID"=>"944219","kingdom"=>"Animalia","phylum"=>"Arthropoda","class"=>"Diplopoda","order"=>"Callipodida","family"=>"Abacionidae","genus"=>"Abacion","lsid"=>"urn:lsid:marinespecies.org:taxname:944219","isMarine"=>"0","isBrackish"=>"0","isFreshwater"=>"0","isTerrestrial"=>"1","isExtinct"=>"None","match_type"=>"exact","modified"=>"2017-01-20T10:14:02Z"'}

{'_execution_time': 0.069, 'affected_rows': 1, 'success': True, 'auth_check': {'success': True, 'session': None, 'auth_level': None}}


{'id': 20382,
 'itisNameWInd': None,
 'matchMethod': 'Not Matched',
 'matchString': 'Abagrotis barnesi',
 'scientificname_search': 'Abagrotis barnesi',
 'scientificname_submitted': 'Abagrotis barnesi',
 'wormsPairs': '"cacheDate"=>"2017-06-26T13:53:52.939201","wormsMatchMethod"=>"Not Matched"'}

...

{'_execution_time': 0.067, 'affected_rows': 1, 'success': True, 'auth_check': {'success': True, 'session': None, 'auth_level': None}}


{'id': 20508,
 'itisNameWInd': 'Acroneuria arenosa',
 'matchMethod': 'Not Matched',
 'matchString': 'Acroneuria arenosa',
 'scientificname_search': 'Acroneuria arenosa',
 'scientificname_submitted': 'Acroneuria arenosa',
 'wormsPairs': '"cacheDate"=>"2017-06-26T13:55:22.191710","wormsMatchMethod"=>"Not Matched"'}

{'_execution_time': 0.067, 'affected_rows': 1, 'success': True, 'auth_check': {'success': True, 'session': None, 'auth_level': None}}


{'id': 20511,
 'itisNameWInd': 'Acroneuria flinti',
 'matchMethod': 'Not Matched',
 'matchString': 'Acroneuria flinti',
 'scientificname_search': 'Acroneuria flinti',
 'scientificname_submitted': 'Acroneuria flinti',
 'wormsPairs': '"cacheDate"=>"2017-06-26T13:55:24.178278","wormsMatchMethod"=>"Not Matched"'}

{'_execution_time': 0.064, 'affected_rows': 1, 'success': True, 'auth_check': {'success': True, 'session': None, 'auth_level': None}}
